# @FutPythonTrader

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
# from google.colab import data_table
# data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')
from pycaret.classification import *
import pycaret 
def dia():
    from datetime import date, timedelta
    dia = date.today() - timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

# Pycaret

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import *

# Base de Dados e Jogos do Dia

In [ ]:
base = pd.read_csv("https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/futpythontraderpunter.csv?raw=true")
base["Date"] = pd.to_datetime(base["Date"])
base['Date'] = base['Date'].dt.date
base = base.sort_values('Date')
base = base[['Date','League','Season','Home','Away','FT_Goals_H','FT_Goals_A','FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']]

In [ ]:
################################################################################
# Probabilidades
################################################################################
base['p_H'] = 1 / base.FT_Odd_H
base['p_D'] = 1 / base.FT_Odd_D
base['p_A'] = 1 / base.FT_Odd_A
base['p_Over'] = 1 / base.FT_Odd_Over25
base['p_Under'] = 1 / base.FT_Odd_Under25
################################################################################

################################################################################
# CV das Odds do Match Odds
################################################################################
base['CV_HDA'] = base[['p_H','p_D','p_A']].std(ddof=0, axis=1) / base[['p_H','p_D','p_A']].mean(axis=1)
################################################################################

################################################################################
# Valor do Gol
################################################################################
base['VG_H'] = base.FT_Goals_H * base.p_A
base['VG_A'] = base.FT_Goals_A * base.p_H
base['Media_VG_H'] = base.groupby('Home')['VG_H'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['Media_VG_A'] = base.groupby('Away')['VG_A'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['Media_VG_H'] = base.groupby('Home')['Media_VG_H'].shift(1)
base['Media_VG_A'] = base.groupby('Away')['Media_VG_A'].shift(1)
base['DP_VG_H'] = base.groupby('Home')['VG_H'].rolling(window=5, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DP_VG_A'] = base.groupby('Away')['VG_A'].rolling(window=5, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DP_VG_H'] = base.groupby('Home')['DP_VG_H'].shift(1)
base['DP_VG_A'] = base.groupby('Away')['DP_VG_A'].shift(1)
base['CV_VG_H'] = base['DP_VG_H'] / base['Media_VG_H']
base['CV_VG_A'] = base['DP_VG_A'] / base['Media_VG_A']
################################################################################

################################################################################
# Custo do Gol
################################################################################
base['CG_H'] = base['p_H'] / base['FT_Goals_H']
base['CG_A'] = base['p_A'] / base['FT_Goals_A']
base['Media_CG_H'] = base.groupby('Home')['CG_H'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['Media_CG_A'] = base.groupby('Away')['CG_A'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['Media_CG_H'] = base.groupby('Home')['Media_CG_H'].shift(1)
base['Media_CG_A'] = base.groupby('Away')['Media_CG_A'].shift(1)
base['DP_CG_H'] = base.groupby('Home')['CG_H'].rolling(window=5, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DP_CG_A'] = base.groupby('Away')['CG_A'].rolling(window=5, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DP_CG_H'] = base.groupby('Home')['DP_CG_H'].shift(1)
base['DP_CG_A'] = base.groupby('Away')['DP_CG_A'].shift(1)
base['CV_CG_H'] = base['DP_CG_H'] / base['Media_CG_H']
base['CV_CG_A'] = base['DP_CG_A'] / base['Media_CG_A']
################################################################################

base = base.drop(['VG_H', 'VG_A', 'CG_H', 'CG_A'], axis=1)

base.replace(np.inf, 1, inplace=True)
base.dropna(inplace=True)
base.reset_index(inplace=True, drop=True)
base.index = base.index.set_names(['Nº'])
base = base.rename(index=lambda x: x + 1)

In [ ]:
ligas = base.sort_values(['League'])
ligas = ligas['League'].unique()
ligas

In [ ]:
liga = 'ARGENTINA - LIGA PROFESIONAL'

In [ ]:
flt = base.League == liga
df = base[flt]
df = drop_reset_index(df)

In [ ]:
df

In [ ]:
df.columns.to_list()

# Criação do Modelo no Pycaret

In [ ]:
# Over 2.5
df.loc[((df['FT_Goals_H'] + df['FT_Goals_A']) > 2), 'Over25'] = 1
df.loc[((df['FT_Goals_H'] + df['FT_Goals_A']) < 3), 'Over25'] = 0

In [ ]:
features = ['FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No','CV_HDA',
            'Media_VG_H','Media_VG_A','DP_VG_H','DP_VG_A','CV_VG_H','CV_VG_A',
            'Media_CG_H','Media_CG_A','DP_CG_H','DP_CG_A','CV_CG_H','CV_CG_A']

label = 'Over25'

In [ ]:
# Configurando o ambiente PyCaret
exp_clf = setup(data = df[features], target = df[label])

In [ ]:
# Comparando diferentes modelos de classificação
best_model = compare_models()

In [ ]:
# Treinando o melhor modelo com os dados de treino
final_model = finalize_model(best_model)

In [ ]:
# Fazendo previsões com os dados de teste
predictions = predict_model(final_model)

# Resultados

In [ ]:
flt = predictions.prediction_label == 1
Over = predictions[flt]
Over.loc[(Over['Over25'] == 1), 'Profit'] = Over['FT_Odd_Over25']-1
Over.loc[(Over['Over25'] == 0), 'Profit'] = -1
grafico(Over,'Over 2.5 Pycaret')

# Salvando o Modelo

In [ ]:
# Salvando o modelo treinado
save_model(final_model, 'Pycaret '+liga)

# Importando os Jogos do Dia

In [ ]:
jogos_do_dia = pd.read_csv('https://github.com/futpythontrader/YouTube/blob/main/Jogos_do_Dia_FlashScore/'+str(dia())+'_Jogos_do_Dia_FlashScore.csv?raw=true')
jogos_do_dia = jogos_do_dia[['League','Round','Date','Time','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']]
Jogos_do_Dia = drop_reset_index(jogos_do_dia)
Jogos_do_Dia

In [ ]:
base = pd.read_csv("https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/futpythontraderpunter.csv?raw=true")
base["Date"] = pd.to_datetime(base["Date"])
base['Date'] = base['Date'].dt.date
base = base.sort_values('Date')
base = base[['Date','League','Season','Home','Away','FT_Goals_H','FT_Goals_A','FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']]

In [ ]:
################################################################################
# Probabilidades
################################################################################
base['p_H'] = 1 / base.FT_Odd_H
base['p_D'] = 1 / base.FT_Odd_D
base['p_A'] = 1 / base.FT_Odd_A
base['p_Over'] = 1 / base.FT_Odd_Over25
base['p_Under'] = 1 / base.FT_Odd_Under25
################################################################################

################################################################################
# CV das Odds do Match Odds
################################################################################
base['CV_HDA'] = base[['p_H','p_D','p_A']].std(ddof=0, axis=1) / base[['p_H','p_D','p_A']].mean(axis=1)
################################################################################
# Pontos
base.loc[base['FT_Goals_H'] > base['FT_Goals_A'], 'Ptos_H'] = 3
base.loc[base['FT_Goals_H'] == base['FT_Goals_A'], 'Ptos_H'] = 1
base.loc[base['FT_Goals_H'] < base['FT_Goals_A'], 'Ptos_H'] = 0

base.loc[base['FT_Goals_H'] < base['FT_Goals_A'], 'Ptos_A'] = 3
base.loc[base['FT_Goals_H'] == base['FT_Goals_A'], 'Ptos_A'] = 1
base.loc[base['FT_Goals_H'] > base['FT_Goals_A'], 'Ptos_A'] = 0

base["Ptos_H"] = base["Ptos_H"].astype('int64')
base["Ptos_A"] = base["Ptos_A"].astype('int64')

base['Media_Ptos_H'] = base.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
base['Media_Ptos_A'] = base.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

base['DesvPad_Ptos_H'] = base.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DesvPad_Ptos_A'] = base.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

base['CV_Ptos_H'] = base['DesvPad_Ptos_H'] / base['Media_Ptos_H']
base['CV_Ptos_A'] = base['DesvPad_Ptos_A'] / base['Media_Ptos_A']

################################################################################
# Valor do Gol
################################################################################
base['VG_H'] = base.FT_Goals_H * base.p_A
base['VG_A'] = base.FT_Goals_A * base.p_H
base['Media_VG_H'] = base.groupby('Home')['VG_H'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['Media_VG_A'] = base.groupby('Away')['VG_A'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['DP_VG_H'] = base.groupby('Home')['VG_H'].rolling(window=5, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DP_VG_A'] = base.groupby('Away')['VG_A'].rolling(window=5, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['CV_VG_H'] = base['DP_VG_H'] / base['Media_VG_H']
base['CV_VG_A'] = base['DP_VG_A'] / base['Media_VG_A']
################################################################################

################################################################################
# Custo do Gol
################################################################################
base['CG_H'] = base['p_H'] / base['FT_Goals_H']
base['CG_A'] = base['p_A'] / base['FT_Goals_A']
base['CG_H'] = base['CG_H'].replace([np.inf, -np.inf, 0], 1)
base['CG_A'] = base['CG_A'].replace([np.inf, -np.inf, 0], 1)
base['Media_CG_H'] = base.groupby('Home')['CG_H'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['Media_CG_A'] = base.groupby('Away')['CG_A'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
base['DP_CG_H'] = base.groupby('Home')['CG_H'].rolling(window=11, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DP_CG_A'] = base.groupby('Away')['CG_A'].rolling(window=11, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['CV_CG_H'] = base['DP_CG_H'] / base['Media_CG_H']
base['CV_CG_A'] = base['DP_CG_A'] / base['Media_CG_A']
################################################################################

base = base.drop(['VG_H', 'VG_A', 'CG_H', 'CG_A'], axis=1)

base.replace(np.inf, 1, inplace=True)
base.dropna(inplace=True)
base.reset_index(inplace=True, drop=True)
base.index = base.index.set_names(['Nº'])
base = base.rename(index=lambda x: x + 1)

base_H = base[['Home','Media_VG_H','DP_VG_H','CV_VG_H','Media_CG_H','DP_CG_H','CV_CG_H','Media_Ptos_H','DesvPad_Ptos_H']]

base_A = base[['Away','Media_VG_A','DP_VG_A','CV_VG_A','Media_CG_A','DP_CG_A','CV_CG_A', 'Media_Ptos_A', 'DesvPad_Ptos_A']]

In [ ]:
lista=[]

for a,b,c,d,e,f,g,h,i,j,k,l in zip(Jogos_do_Dia.League,
                                   Jogos_do_Dia.Date,
                                   Jogos_do_Dia.Time,
                                   Jogos_do_Dia.Home,
                                   Jogos_do_Dia.Away,
                                   Jogos_do_Dia.FT_Odd_H,
                                   Jogos_do_Dia.FT_Odd_D,
                                   Jogos_do_Dia.FT_Odd_A,
                                   Jogos_do_Dia.FT_Odd_Over25,
                                   Jogos_do_Dia.FT_Odd_Under25,
                                   Jogos_do_Dia.FT_Odd_BTTS_Yes,
                                   Jogos_do_Dia.FT_Odd_BTTS_No):
        League = a
        Date = b
        Time = c
        home = d
        away = e
        FT_Odd_H = f
        FT_Odd_D = g
        FT_Odd_A = h
        FT_Odd_Over25 = i
        FT_Odd_Under25 = j
        FT_Odd_BTTS_Yes = k
        FT_Odd_BTTS_No = l

        df1 = base_H[base_H.Home == home].tail(1)

        df2 = base_A[base_A.Away == away].tail(1)

        jogo = {}

        jogo["League"] = League
        jogo["Date"] = Date
        jogo["Time"] = Time
        jogo["Home"] = home
        jogo["Away"] = away
        jogo["FT_Odd_H"] = FT_Odd_H
        jogo["FT_Odd_D"] = FT_Odd_D
        jogo["FT_Odd_A"] = FT_Odd_A
        jogo["FT_Odd_Over25"] = FT_Odd_Over25
        jogo["FT_Odd_Under25"] = FT_Odd_Under25
        jogo["FT_Odd_BTTS_Yes"] = FT_Odd_BTTS_Yes
        jogo["FT_Odd_BTTS_No"] = FT_Odd_BTTS_No

        try:
            jogo['Media_VG_H'] = df1[df1.Home == home]['Media_VG_H'].iloc[0]
            jogo['DP_VG_H'] = df1[df1.Home == home]['DP_VG_H'].iloc[0]
            jogo['CV_VG_H'] = df1[df1.Home == home]['CV_VG_H'].iloc[0]

            jogo['Media_CG_H'] = df1[df1.Home == home]['Media_CG_H'].iloc[0]
            jogo['DP_CG_H'] = df1[df1.Home == home]['DP_CG_H'].iloc[0]
            jogo['CV_CG_H'] = df1[df1.Home == home]['CV_CG_H'].iloc[0]

            
            jogo['Media_Ptos_H'] = df1[df1.Home == home]['Media_Ptos_H'].iloc[0]
            jogo['DesvPad_Ptos_H'] = df1[df1.Home == home]['DesvPad_Ptos_H'].iloc[0]
            
            jogo['Media_VG_A'] = df2[df2.Away == away]['Media_VG_A'].iloc[0]
            jogo['DP_VG_A'] = df2[df2.Away == away]['DP_VG_A'].iloc[0]
            jogo['CV_VG_A'] = df2[df2.Away == away]['CV_VG_A'].iloc[0]

            jogo['Media_CG_A'] = df2[df2.Away == away]['Media_CG_A'].iloc[0]
            jogo['DP_CG_A'] = df2[df2.Away == away]['DP_CG_A'].iloc[0]
            jogo['CV_CG_A'] = df2[df2.Away == away]['CV_CG_A'].iloc[0]
            
            jogo['Media_Ptos_A'] = df2[df2.Away == away]['Media_Ptos_A'].iloc[0]
            jogo['DesvPad_Ptos_A'] = df2[df2.Away == away]['DesvPad_Ptos_A'].iloc[0]
            
            
            
        except:
            pass

        lista.append(jogo)


df = pd.DataFrame(lista)
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
################################################################################
# Probabilidades
################################################################################
df['p_H'] = 1 / df.FT_Odd_H
df['p_D'] = 1 / df.FT_Odd_D
df['p_A'] = 1 / df.FT_Odd_A
df['p_Over'] = 1 / df.FT_Odd_Over25
df['p_Under'] = 1 / df.FT_Odd_Under25

################################################################################
# CV das Odds do Match Odds
################################################################################
df['CV_HDA'] = df[['p_H','p_D','p_A']].std(ddof=0, axis=1) / df[['p_H','p_D','p_A']].mean(axis=1)

df['coeficiente_pontos'] = df['Media_Ptos_H'] - df['Media_Ptos_A']
df['cv_pontos'] = df['DesvPad_Ptos_H'] + df['DesvPad_Ptos_A']


# Utilizando o Modelo para Novos Dados

In [ ]:
flt = df.League == 'ARGENTINA - LIGA PROFESIONAL'
df = df[flt]
df = drop_reset_index(df)
df

In [ ]:
# Carregando o modelo salvo
saved_final_model = load_model('LAYAWAY_ARKAD_ARGENTINA - LIGA PROFESIONAL')

# Fazendo previsões com os novos dados
new_predictions = predict_model(saved_final_model, data=df)
print()
display(new_predictions)